In [43]:
import json
import requests
import sys
import re
import pandas as pd
from pprint import pprint
from datetime import datetime, timedelta 

API_KEY = ''
BASE_URL = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline'
unit_group = 'metric'

In [59]:
# Returns the weather forecast for the following 12 hours
def get_12hr_forecast(location: str) -> list:
    
    date_start = datetime.now().replace(minute=0, second=0)
    date_end = date_start + timedelta(hours=12)
    dt_format = '%Y-%m-%dT%H:%M:%S'
    date_start, date_end = date_start.strftime(dt_format), date_end.strftime(dt_format) 

    api_query = f'{BASE_URL}/{location}/{date_start}/{date_end}?unitGroup={unit_group}&key={API_KEY}&contentType=json'

    response = requests.request("GET", url=api_query)
    if response.status_code != 200:
        print('Unexpected Status code: ', response.status_code)
        sys.exit() 
    data = strip_irrelevant_data(response.json(), date_start, date_end)
    
    return data


# Deletes the json entries outside of the relevant 12-hour forecast timeframe
def strip_irrelevant_data(data: dict, date_start: str, date_end: str) -> list:
    
    if len(data['days']) == 1:
        data = data['days'][0]['hours'][datetime.strptime(date_start, '%Y-%m-%dT%H:%M:%S').time().hour : datetime.strptime(date_end, '%Y-%m-%dT%H:%M:%S').time().hour]  
    else:
        data = data['days'][0]['hours'][datetime.strptime(date_start, '%Y-%m-%dT%H:%M:%S').time().hour :] + data['days'][1]['hours'][: datetime.strptime(date_end, '%Y-%m-%dT%H:%M:%S').time().hour] 
    
    return data

In [60]:
data = get_12hr_forecast('Kyiv, Ukraine')
data

[{'datetime': '21:00:00',
  'datetimeEpoch': 1679511600,
  'temp': 9.1,
  'feelslike': 9.1,
  'humidity': 66.08,
  'dew': 3.1,
  'precip': 0.0,
  'precipprob': 0.0,
  'snow': 0.0,
  'snowdepth': 0.0,
  'preciptype': None,
  'windgust': 2.9,
  'windspeed': 1.1,
  'winddir': 106.6,
  'pressure': 1017.0,
  'visibility': 24.1,
  'cloudcover': 0.0,
  'solarradiation': 0.0,
  'solarenergy': None,
  'uvindex': 0.0,
  'severerisk': 10.0,
  'conditions': 'Clear',
  'icon': 'clear-night',
  'stations': None,
  'source': 'fcst'},
 {'datetime': '22:00:00',
  'datetimeEpoch': 1679515200,
  'temp': 8.5,
  'feelslike': 8.5,
  'humidity': 67.85,
  'dew': 2.9,
  'precip': 0.0,
  'precipprob': 0.0,
  'snow': 0.0,
  'snowdepth': 0.0,
  'preciptype': None,
  'windgust': 2.9,
  'windspeed': 1.8,
  'winddir': 158.1,
  'pressure': 1017.0,
  'visibility': 24.1,
  'cloudcover': 0.0,
  'solarradiation': 0.0,
  'solarenergy': None,
  'uvindex': 0.0,
  'severerisk': 10.0,
  'conditions': 'Clear',
  'icon': 'clear

In [61]:
df = pd.DataFrame(data)
df = df.set_index('datetime')
df

,datetimeEpoch,temp,feelslike,humidity,dew,precip,precipprob,snow,snowdepth,preciptype,...,visibility,cloudcover,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations,source
datetime,,,,,,,,,,,,,,,,,,,,,
21:00:00,1679511600,9.1,9.1,66.08,3.1,0.0,0.0,0.0,0.0,None,...,24.1,0.0,0.0,NaN,0.0,10.0,Clear,clear-night,None,fcst
22:00:00,1679515200,8.5,8.5,67.85,2.9,0.0,0.0,0.0,0.0,None,...,24.1,0.0,0.0,NaN,0.0,10.0,Clear,clear-night,None,fcst
23:00:00,1679518800,8.2,8.2,67.78,2.6,0.0,0.0,0.0,0.0,None,...,24.1,0.0,0.0,NaN,0.0,10.0,Clear,clear-night,None,fcst
00:00:00,1679522400,7.8,7.8,69.16,2.5,0.0,0.0,0.0,0.0,None,...,24.1,0.0,0.0,NaN,0.0,10.0,Clear,clear-night,None,fcst
01:00:00,1679526000,7.4,7.4,71.07,2.5,0.0,0.0,0.0,0.0,None,...,24.1,0.0,0.0,NaN,0.0,10.0,Clear,clear-night,None,fcst
02:00:00,1679529600,7.0,7.0,73.57,2.6,0.0,0.0,0.0,0.0,None,...,24.1,0.0,0.0,NaN,0.0,10.0,Clear,clear-night,None,fcst
03:00:00,1679533200,6.6,6.6,75.61,2.6,0.0,0.0,0.0,0.0,None,...,24.1,0.0,0.0,NaN,0.0,10.0,Clear,clear-night,None,fcst
04:00:00,1679536800,6.2,6.2,77.73,2.6,0.0,0.0,0.0,0.0,None,...,24.1,0.0,0.0,NaN,0.0,10.0,Clear,clear-night,None,fcst
05:00:00,1679540400,5.9,5.9,78.79,2.5,0.0,0.0,0.0,0.0,None,...,24.1,0.0,0.0,NaN,0.0,10.0,Clear,clear-night,None,fcst
